In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

conn = sqlite3.connect('exp5.db') # Change to the appropriate database file for different experiments

rooms = pd.read_sql_query("""
    SELECT DISTINCT room_id FROM anomalies WHERE timestamp LIKE '2025-06-03%' ORDER BY room_id ASC
""", conn, parse_dates=['timestamp'])

print (f"Rooms with anomalies: {(rooms.room_id.unique())}")

lstm_ids = pd.read_sql_query("""
    SELECT DISTINCT room_id FROM injected_drift ORDER BY room_id ASC
""", conn, parse_dates=['timestamp'])

print(f"LSTM room IDs: {(lstm_ids.room_id.unique())}")

room_ids = rooms.room_id.unique()
ids_str = ", ".join(map(str, room_ids))
lstm_ids_str = ", ".join(map(str, lstm_ids.room_id.unique()))
grid_rooms = set(room_ids) | set(lstm_ids.room_id.unique())
grid_rooms_str = ", ".join(map(str, sorted(grid_rooms)))
print(f"Grid rooms: {grid_rooms_str}")
print(f"Modified room IDs: {ids_str}")
print(f"LSTM room IDs: {lstm_ids_str}")

# Baseline

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

conn_base = sqlite3.connect('exp_baseline.db')

naia_df_base = pd.read_sql_query(f"""
    SELECT *
    FROM anomalies
    WHERE source_model = 'NAIA'
      AND room_id IN ({grid_rooms_str})
""", conn_base, parse_dates=['timestamp'])

lstm_df_base = pd.read_sql_query(f"""
    SELECT *
    FROM anomalies
    WHERE source_model = 'LSTM-AE'
      AND room_id IN ({lstm_ids_str})
""", conn_base, parse_dates=['timestamp'])

conn_base.close()

for df in (naia_df_base, lstm_df_base):
    df['hour'] = df['timestamp'].dt.hour
    df['date'] = df['timestamp'].dt.date

last_date_naia_base = naia_df_base['date'].max()
last_date_lstm_base = lstm_df_base['date'].max()

valid_dates_base = [d for d in (last_date_naia_base, last_date_lstm_base) if pd.notnull(d)]
if not valid_dates_base:
    raise RuntimeError("No anomalies found for either NAIA or LSTM_AE in baseline.")
last_date_base = max(valid_dates_base)

naia_last_base = naia_df_base[naia_df_base['date'] == last_date_base]
lstm_last_base = lstm_df_base[lstm_df_base['date'] == last_date_base]

rooms_naia_base = set(naia_last_base['room_id'].unique())
rooms_lstm_base = set(lstm_last_base['room_id'].unique())
all_rooms_base = sorted(rooms_naia_base.union(rooms_lstm_base).union(grid_rooms))

all_hours_base = list(range(24))

grid_naia_base = pd.DataFrame(0, index=all_hours_base, columns=all_rooms_base)
grid_lstm_base = pd.DataFrame(0, index=all_hours_base, columns=all_rooms_base)

for _, row in naia_last_base.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_naia_base.at[hr, rm] = 1

for _, row in lstm_last_base.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_lstm_base.at[hr, rm] = 1

grid_both_base = (grid_naia_base & grid_lstm_base).astype(int)

grid_either_base = (grid_naia_base | grid_lstm_base).astype(int)

print("Grid NAIA")
print(grid_naia_base)
print("Grid LSTM_AE")
print(grid_lstm_base)
print("Grid Both Models")
print(grid_both_base)
print("Grid Either Model")
print(grid_either_base)

# -----------------------
# Plot: NAIA anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_naia_base.values, aspect='auto', cmap='Reds', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms_base)), labels=all_rooms_base, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours_base)
plt.title(f'NAIA Anomalous Hours per Room (Last Date = {last_date_base})')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: LSTM_AE anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_lstm_base.values, aspect='auto', cmap='Blues', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms_base)), labels=all_rooms_base, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours_base)
plt.title(f'LSTM_AE Anomalous Hours per Room (Last Date = {last_date_base})')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Anomalous hours flagged by BOTH models
plt.figure(figsize=(12, 4))
plt.imshow(grid_both_base.values, aspect='auto', cmap='Purples', origin='lower')
plt.colorbar(label='Anomaly by Both (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms_base)), labels=all_rooms_base, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours_base)
plt.title(f'Anomalous Hours Flagged by BOTH Models (Last Date = {last_date_base})')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Anomalies flagged by either model
plt.figure(figsize=(12, 4))
plt.imshow(grid_either_base.values, aspect='auto', cmap='Greys', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms_base)), labels=all_rooms_base, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours_base)
plt.title(f'Anomalous Hours Flagged by Either Model (Last Date = {last_date_base})')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# results statistics
def print_statistics(grid, model_name):
    total_anomalies = grid.values.sum()
    total_rooms = grid.shape[1]
    total_hours = grid.shape[0]
    total_either = grid_either_base.values.sum()

    
    print(f"\nStatistics for {model_name}:")
    print(f"Total Anomalies Detected: {total_anomalies}")
    print(f"Total Rooms Monitored: {total_rooms}")
    print(f"Total Hours Monitored: {total_hours}")
    print(f"Average Anomalies per Room: {total_anomalies / total_rooms:.2f}")
    print(f"Average Anomalies per Hour: {total_anomalies / total_hours:.2f}")
    print(f"Total Anomalies Detected by Either Model: {total_either}")
    print(f"Percentage of Anomalies detected by {model_name}: "f"{(total_anomalies / total_either) * 100:.2f}%")
# Print statistics for each grid

print_statistics(grid_naia_base, "NAIA Model Baseline")
print_statistics(grid_lstm_base, "LSTM-AE Model Baseline")
print_statistics(grid_both_base, "Both Models Combined Baseline")
print_statistics(grid_either_base, "Either Model Detected Baseline")

# Final results (graphs)

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

conn = sqlite3.connect('exp5.db')

naia_df = pd.read_sql_query("""
    SELECT * FROM anomalies
    WHERE source_model = 'NAIA'
""", conn, parse_dates=['timestamp'])

lstm_df = pd.read_sql_query("""
    SELECT * FROM anomalies
    WHERE source_model = 'LSTM-AE'
""", conn, parse_dates=['timestamp'])

data_df = pd.read_sql_query("""
    SELECT * FROM injected_drift
""", conn, parse_dates=['timestamp'])

conn.close()

for df in (naia_df, lstm_df, data_df):
    df['hour'] = df['timestamp'].dt.hour
    df['date'] = df['timestamp'].dt.date

last_date_naia = naia_df['date'].max()
last_date_lstm = lstm_df['date'].max()
last_date_data = data_df['date'].max()

valid_dates = [d for d in (last_date_naia, last_date_lstm, last_date_data) if pd.notnull(d)]
if not valid_dates:
    raise RuntimeError("No anomalies found for either NAIA or LSTM_AE or data injection.")
last_date = max(valid_dates)

naia_last = naia_df[naia_df['date'] == last_date]
lstm_last = lstm_df[lstm_df['date'] == last_date]
data_last = data_df[data_df['date'] == last_date]

rooms_naia = set(naia_last['room_id'].unique())
rooms_lstm = set(lstm_last['room_id'].unique())
rooms_data = set(data_last['room_id'].unique())
all_rooms = sorted(rooms_naia.union(rooms_lstm).union(rooms_data))

all_hours = list(range(24))

grid_naia = pd.DataFrame(0, index=all_hours, columns=all_rooms)
grid_lstm = pd.DataFrame(0, index=all_hours, columns=all_rooms)
grid_data = pd.DataFrame(0, index=all_hours, columns=all_rooms)

for _, row in naia_last.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_naia.at[hr, rm] = 1

for _, row in lstm_last.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_lstm.at[hr, rm] = 1

for _, row in data_last.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_data.at[hr, rm] = 1


grid_both = (grid_naia & grid_lstm).astype(int)

grid_inject_detected = (grid_both & grid_data).astype(int)

grid_injected_naia = (grid_naia & grid_data).astype(int)
grid_injected_lstm = (grid_lstm & grid_data).astype(int)

grid_base_not_detected_naia = (grid_naia_base & ~grid_naia).astype(int)
grid_base_not_detected_lstm = (grid_lstm_base & ~grid_lstm).astype(int)

grid_naia_false_positive = (grid_naia & ~grid_data & ~grid_naia_base).astype(int)
grid_lstm_false_positive = (grid_lstm & ~grid_data & ~grid_lstm_base).astype(int)
grid_either_false_positive = (grid_naia_false_positive | grid_lstm_false_positive).astype(int)

grid_either = ((grid_naia | grid_lstm) & grid_data).astype(int)

grid_undetected = (grid_data & ~(grid_naia | grid_lstm)).astype(int)

grid_detected_not_injected = ((grid_naia | grid_lstm) & ~grid_data).astype(int)

grid_false_negatives = (grid_detected_not_injected & ~grid_either_base).astype(int)

print("Grid NAIA")
print(grid_naia)
print("Grid LSTM_AE")
print(grid_lstm)
print("Grid Both Models")
print(grid_both)
print("Grid Data Injection Detected by Both Models")
print(grid_inject_detected)
print("Grid Data Injection NAIA")
print(grid_injected_naia)
print("Grid Data Injection LSTM_AE")
print(grid_injected_lstm)
print("Grid Base Not Detected NAIA")
print(grid_base_not_detected_naia)
print("Grid Base Not Detected LSTM_AE")
print(grid_base_not_detected_lstm)
print("Grid NAIA False Positive")
print(grid_naia_false_positive)
print("Grid LSTM_AE False Positive")
print(grid_lstm_false_positive)
print("Grid Data Injection")
print(grid_data)
print("Grid Either Model")
print(grid_either)
print("Grid Undetected by Either Model")
print(grid_undetected)
print("Grid Detected but not Injected")
print(grid_detected_not_injected)
print("Grid False Negatives")
print(grid_false_negatives)


# -----------------------
# Plot: NAIA anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_naia.values, aspect='auto', cmap='Reds', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours per room detected by NAIA on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: LSTM_AE anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_lstm.values, aspect='auto', cmap='Blues', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours per room detected by LSTM-AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Anomalous hours flagged by BOTH models
plt.figure(figsize=(12, 4))
plt.imshow(grid_both.values, aspect='auto', cmap='Purples', origin='lower')
plt.colorbar(label='Anomaly by Both (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours detected by NAIA AND LSTM-AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Data injection anomalies detected by both models
plt.figure(figsize=(12, 4))
plt.imshow(grid_inject_detected.values, aspect='auto', cmap='Oranges', origin='lower')
plt.colorbar(label='Anomaly Detected by Both (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Data Injection Anomalous Hours Detected by BOTH Models (Last Date = {last_date})')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()


# -----------------------
# Plot: Anomalies flagged by either model
plt.figure(figsize=(12, 4))
plt.imshow(grid_either.values, aspect='auto', cmap='Greys', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours detected by NAIA OR LSTM-AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Data injection anomalies detected by NAIA model
plt.figure(figsize=(12, 4))
plt.imshow(grid_injected_naia.values, aspect='auto', cmap='Oranges', origin='lower')
plt.colorbar(label='Anomaly Detected by NAIA (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Data injection anomalous hours detected by NAIA on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Data injection anomalies detected by LSTM_AE model
plt.figure(figsize=(12, 4))
plt.imshow(grid_injected_lstm.values, aspect='auto', cmap='Oranges', origin='lower')
plt.colorbar(label='Anomaly Detected by LSTM_AE (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Data injection anomalous hours detected by LSTM_AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Baseline anomalies not detected by NAIA model
plt.figure(figsize=(12, 4))
plt.imshow(grid_base_not_detected_naia.values, aspect='auto', cmap='Reds', origin='lower')
plt.colorbar(label='Anomaly NOT Detected by NAIA (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Baseline anomalous hours NOT detected by NAIA on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Baseline anomalies not detected by LSTM_AE model
plt.figure(figsize=(12, 4))
plt.imshow(grid_base_not_detected_lstm.values, aspect='auto', cmap='Blues', origin='lower')
plt.colorbar(label='Anomaly NOT Detected by LSTM_AE (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Baseline anomalous hours NOT detected by LSTM_AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: NAIA model false positives
plt.figure(figsize=(12, 4))
plt.imshow(grid_naia_false_positive.values, aspect='auto', cmap='Reds', origin='lower')
plt.colorbar(label='False Positive (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'NAIA Model False Positives on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: LSTM_AE model false positives
plt.figure(figsize=(12, 4))
plt.imshow(grid_lstm_false_positive.values, aspect='auto', cmap='Blues', origin='lower')
plt.colorbar(label='False Positive (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'LSTM_AE Model False Positives on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Data injection anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_data.values, aspect='auto', cmap='Greens', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours injected per Room on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()


# -----------------------
# Plot: Undetected anomalies by either model
plt.figure(figsize=(12, 4))
plt.imshow(grid_undetected.values, aspect='auto', cmap='Blues', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours NOT detected by either model on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Detected anomalies not injected
plt.figure(figsize=(12, 4))
plt.imshow(grid_detected_not_injected.values, aspect='auto', cmap='Purples', origin='lower')
plt.colorbar(label='Anomaly Detected but NOT Injected (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Non-injected anomalous hours detected on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: False Negatives
plt.figure(figsize=(12, 4))
plt.imshow(grid_false_negatives.values, aspect='auto', cmap='Greens', origin='lower')
plt.colorbar(label='False Negative (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'False Negatives on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()





# Final results (statistics)

In [ ]:
# results statistics
def print_statistics(grid, model_name):
    total_anomalies = grid.values.sum()
    total_rooms = grid.shape[1]
    total_hours = grid.shape[0]
    total_injected = grid_data.values.sum()

    
    print(f"\nStatistics for {model_name}:")
    print(f"Total Anomalies Detected: {total_anomalies}")
    print(f"Total Rooms Monitored: {total_rooms}")
    print(f"Total Hours Monitored: {total_hours}")
    print(f"Average Anomalies per Room: {total_anomalies / total_rooms:.2f}")
    print(f"Average Anomalies per Hour: {total_anomalies / total_hours:.2f}")
    print(f"Percentage of Anomalies Detected: {total_anomalies / total_injected * 100:.2f}%")


print(f"Overall Statistics:")
total_injected = grid_data.values.sum()
print(f"Total Anomalous Hours Injected: {total_injected}")
print(f"Injected anomalies detected by NAIA: {grid_injected_naia.values.sum()}")
print(f"Injected anomalies detected by LSTM-AE: {grid_injected_lstm.values.sum()}")
print(f"Injected anomalies detected by both models: {grid_inject_detected.values.sum()}")
print(f"Injected anomalies detected by either model: {grid_either.values.sum()}")
print(f"Number of undetected injected anomalies: {grid_undetected.values.sum()}")
print(f"NAIA of False Positives percentage: {((grid_naia_false_positive.values.sum() ) / grid_naia.values.sum()) * 100:.2f}%")
print(f"Number of NAIA False Positives: {grid_naia_false_positive.values.sum()}")
print(f"LSTM-AE of False Positives percentage: {((grid_lstm_false_positive.values.sum() ) / grid_lstm.values.sum()) * 100:.2f}%")
print(f"Number of LSTM-AE False Positives: {grid_lstm_false_positive.values.sum()}")
print(f"Overall (either two-model) False Positives percentage: {((grid_naia_false_positive.values.sum() + grid_lstm_false_positive.values.sum()) / (grid_naia.values.sum() + grid_lstm.values.sum())) * 100:.2f}%")
print(f"Number of either model False Positives: {grid_either_false_positive.values.sum()}")
print(f"Percentage of undetected anomalies: {((grid_undetected.values.sum() ) / total_injected) * 100:.2f}%")
print(f"Baseline anomalous hours: {grid_either_base.values.sum()}")
print(f"False negatives: {grid_false_negatives.values.sum()}")
# Print statistics for each grid
print_statistics(grid_naia, "NAIA Model")
print_statistics(grid_lstm, "LSTM-AE Model")
print_statistics(grid_both, "Both Models Combined")
print_statistics(grid_either, "Either Model Detected")
print_statistics(grid_data, "Data Injection")
print_statistics(grid_inject_detected, "Data Injection Detected by Both Models")
print_statistics(grid_injected_naia, "Data Injection Detected by NAIA Model")
print_statistics(grid_injected_lstm, "Data Injection Detected by LSTM-AE Model")
print_statistics(grid_undetected, "Undetected Anomalies")